<a href="https://colab.research.google.com/github/LaloDN/Submarine-sonar-prediction/blob/master/Submarine_sonar_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>